In [48]:
import os
import joblib
print(joblib.__version__)
import pandas as pd
import numpy as np
from sklearn.preprocessing import scale
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from sklearn.preprocessing import LabelEncoder, StandardScaler, MultiLabelBinarizer
from sklearn.metrics import accuracy_score, classification_report
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from xgboost import XGBClassifier
from sklearn.metrics import roc_curve, auc, roc_auc_score
import matplotlib.pyplot as plt
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

1.3.2


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [26]:
def bin_ages(age):
    if pd.isnull(age) or not isinstance(age, (str, float, int)):
        return 'Unknown'
    age_str = str(age)

    if 'pcw' in age_str:
        weeks = int(age_str.split(' ')[0])
        if weeks >= 4 and weeks <= 7: # 4-7 pcw
            return 'Embryonic'
        elif weeks >= 8 and weeks <= 38: # 8-38 pcw
            return 'Prenatal'
    elif 'mos' in age_str or 'M' in age_str:
        months = int(age_str.split(' ')[0])
        if months >= 0 and months <= 19: # 0-19 months
            return 'Infancy'
    elif 'yrs' in age_str or 'Y' in age_str:
        years = int(age_str.split(' ')[0])
        if years >= 1 and years <= 11: # 1-11 years
            return 'Childhood'
        elif years >= 12 and years <= 19: # 12-19 years
            return 'Adolescence'
        elif years >= 20: # 20 years and above
            return 'Adulthood'

    # return 'Unknown' for any other cases
    return 'Unknown'

In [27]:
def normalize_data(data, lower_bound=1, upper_bound=255):
    # apply threshold to filter out very low values
    threshold = data.mean().mean()
    data[data < threshold] = 0

    # scale the data
    max_abs_value = data.abs().max().max()
    data_scaled = data.apply(lambda x: ((x / max_abs_value) * (upper_bound - lower_bound)) + lower_bound if x.max() != 0 else x, axis=1)
    return data_scaled

In [28]:
def convert_to_image(dataframe, image_size=150):
    total_size = image_size * image_size
    image_array = np.zeros((len(dataframe), total_size))

    for i, row in enumerate(dataframe.to_numpy()):
        image_array[i, :len(row)] = row[:total_size]

    # reshape to have it in 2D image format (num_samples, image_width, image_height)
    image_array = image_array.reshape(-1, image_size, image_size)

    return image_array

In [40]:
def preprocess_data(data_path, file_type='csv'):
    """
    Preprocess the data and return the train-test split.

    Args:
    - data_path (str): Path to the data file.
    - file_type (str): File format ('csv', 'excel', 'txt').

    Returns:
    - X_train, X_test, y_train, y_test: Train-test split of the preprocessed data.
    """
    subset_rows = 20


    if 'methylation' in data_path.lower() and file_type == 'csv':
        chunk_size = 5
        chunks = pd.read_csv(data_path, chunksize=chunk_size)

        X_list = []
        y_list = []
        for chunk in chunks:
            chunk = chunk.head(subset_rows)
            X_chunk, y_chunk = preprocess_chunk(chunk, 'Methylation')

            X_list.append(X_chunk)
            y_list.append(y_chunk)

        X_data = pd.concat(X_list, axis=0)
        y_data = pd.concat(y_list, axis=0)

        data = pd.concat([X_data, y_data], axis=1)
        print("Columns after processing all chunks:", data.columns)

    else:
        if file_type == 'csv':
            data = pd.read_csv(data_path, index_col=0)
        elif file_type == 'excel':
            data = pd.read_excel(data_path, index_col=0)
        elif file_type == 'txt':
            data = pd.read_csv(data_path, sep='\t', index_col=0)
        else:
            raise ValueError("Unsupported file type")
        print("Columns after determining data type:", data.columns)

    if 'rnaseq' in data_path.lower():
        data_type = 'RNA-Seq'
    elif 'methylation' in data_path.lower():
        data_type = 'Methylation'
    elif 'microrna' in data_path.lower():
        data_type = 'MicroRNA'
    else:
        raise ValueError("Unknown data type")

    print(data.columns)

    data['age_group'] = data['age'].apply(bin_ages)

    if data_type == 'RNA-Seq':
        label_map = {'Prenatal': 0, 'Infancy': 0, 'Childhood': 1, 'Adolescence': 2, 'Adulthood': 3}
    elif data_type == 'MicroRNA':
        label_map = {'Infancy': 0, 'Childhood': 1, 'Adolescence': 2, 'Adulthood': 3}
    else:
        label_map = {'Embryonic': 0, 'Prenatal': 1, 'Infancy': 2, 'Childhood': 3, 'Adolescence': 4, 'Adulthood': 5}

    data['age_group'] = data['age_group'].map(label_map)

    if data_type == 'RNA-Seq':
        data_numeric = data.drop(['age', 'age_group'], axis=1)
        one_percent_of_samples = data_numeric.shape[1] * 0.01
        mask = data_numeric.gt(1).sum(axis=1) >= one_percent_of_samples
        filtered_data = data[mask]
        X = filtered_data.drop(['age', 'age_group'], axis=1)
        y = filtered_data['age_group']

    elif data_type == 'Methylation':
        relevant_columns = [col for col in data.columns if col.startswith(('cg', 'rs', 'ch'))] + ['age_group']
        filtered_data = data[relevant_columns]
        X = filtered_data.drop(['age_group'], axis=1)
        y = filtered_data['age_group']

    elif data_type == 'MicroRNA':
        relevant_columns = [col for col in data.columns if col.startswith('hsa')] + ['age', 'age_group']
        filtered_data = data[relevant_columns]
        X = filtered_data.drop(['age', 'age_group'], axis=1)
        y = filtered_data['age_group']

    if y.isnull().any():
        print("NaNs present in age_group labels!")
        print(data[data['age_group'].isnull()]['age'])

    print(np.unique(y))
    print("About to shuffle")

    X, y = shuffle(X, y, random_state=0)
    print(np.unique(y))
    # add a small value to avoid log(0)
    X_log_scaled = np.log1p(X)  # log1p applies log(1+x) to avoid log(0)
    max_log_value = X_log_scaled.max().max()
    X_normalized = np.round((X_log_scaled / max_log_value) * 255)
    print(X_normalized.iloc[:5, :10])

    # split the data
    X_train, X_test, y_train, y_test = train_test_split(X_normalized, y, test_size=0.2, random_state=42)

    # convert to images
    X_train_images = convert_to_image(X_train)
    X_test_images = convert_to_image(X_test)

    return X_train_images, X_test_images, y_train, y_test

In [49]:
def save_data_splits(X_train, X_test, y_train, y_test, output_dir):
    """
    Saves the train-test data splits to the specified directory using joblib.

    Args:
    - X_train (pd.DataFrame or np.ndarray): Training data features.
    - X_test (pd.DataFrame or np.ndarray): Testing data features.
    - y_train (pd.Series or np.ndarray): Training data labels.
    - y_test (pd.Series or np.ndarray): Testing data labels.
    - output_dir (str): Directory path where the data splits will be saved.

    Note:
    - If the output directory does not exist, it will be created.
    """
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    joblib.dump(X_train, os.path.join(output_dir, 'X_train_classifiers.pkl'))
    joblib.dump(X_test, os.path.join(output_dir, 'X_test_classifiers.pkl'))
    joblib.dump(y_train, os.path.join(output_dir, 'y_train_classifiers.pkl'))
    joblib.dump(y_test, os.path.join(output_dir, 'y_test_classifiers.pkl'))

In [62]:
def plot_images_and_save(images, output_dir, num_images=5, file_name_prefix='default'):
    """
    Plots a specified number of images and saves the plot.

    Args:
    - images (np.ndarray): Array of images to be plotted.
    - num_images (int): Number of images to plot.
    - output_dir (str): Directory path where the image plot will be saved.
    - file_name_prefix (str): Prefix for the file name of the saved plot.
    """
    fig, axes = plt.subplots(1, num_images, figsize=(20, 4))
    for i, ax in enumerate(axes):
        if i < num_images:
            ax.imshow(images[i], cmap='magma')
            ax.axis('off')
        else:
            ax.axis('off')

    plt.tight_layout()

    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    file_path = os.path.join(output_dir, f"{file_name_prefix}_images.png")
    plt.savefig(file_path)
    plt.close()

    print(f"Images saved to {file_path}")

In [50]:
def create_cnn(input_shape, num_classes):
    model = Sequential([
        Conv2D(32, (3, 3), activation='relu', input_shape=input_shape),
        MaxPooling2D((2, 2)),
        Conv2D(64, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),
        Conv2D(128, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),
        Flatten(),
        Dense(128, activation='relu'),
        Dropout(0.5),
        Dense(num_classes, activation='softmax')
    ])

    return model

In [51]:
def plot_roc_curve(y_test, y_prob, output_dir):
    """
    Plots the Receiver Operating Characteristic (ROC) curve for multi-class classification and saves the plot.

    Args:
    - y_test (pd.Series or np.ndarray): True labels for the test data.
    - y_prob (np.ndarray): Probability estimates of the positive class for each class.
    - output_dir (str): Directory path where the ROC curve plot will be saved.

    Note:
    - The function handles multi-class classification by plotting an ROC curve for each class.
    - If the provided `output_dir` does not exist, it must be created before calling this function.
    """
    n_classes = len(np.unique(y_test))
    mlb = MultiLabelBinarizer(classes=list(range(n_classes)))
    y_test_binarized = mlb.fit_transform(y_test.to_numpy().reshape(-1, 1))

    fpr = dict()
    tpr = dict()
    roc_auc = dict()

    for i in range(n_classes):
        fpr[i], tpr[i], _ = roc_curve(y_test_binarized[:, i], y_prob[:, i])
        roc_auc[i] = auc(fpr[i], tpr[i])

    macro_roc_auc = roc_auc_score(y_test_binarized, y_prob, average='macro')
    for i in range(n_classes):
        print(f"Class {i} ROC AUC: {roc_auc[i]:.2f}")
    print(f"Macro-average ROC AUC: {macro_roc_auc:.2f}")

    colors = ['blue', 'red', 'green', 'orange', 'purple']
    plt.figure(figsize=(10, 6))
    for i, color in zip(range(n_classes), colors):
        plt.plot(fpr[i], tpr[i], color=color, lw=2, label=f'ROC curve (area = {roc_auc[i]:.2f}) for class {i}')
    plt.plot([0, 1], [0, 1], 'k--', lw=2)
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver Operating Characteristic (ROC) Curve')
    plt.legend(loc="lower right")
    plt.savefig(os.path.join(output_dir, 'roc_curve.png'))
    plt.close()

In [63]:
def main(data_paths):
    """
    The main pipeline to preprocess data, train CNNs, evaluate their performance, and save the results.
    """
    for data_path in data_paths:
        data_type = os.path.basename(data_path).split('_')[0]

        X_train, X_test, y_train, y_test = preprocess_data(data_path)
        save_data_splits(X_train, X_test, y_train, y_test, os.path.join('baseline_cnn_classifier_outputs', data_type))

        X_train_images_expanded = np.expand_dims(X_train, axis=-1)
        X_test_images_expanded = np.expand_dims(X_test, axis=-1)
        input_shape = X_train_images_expanded.shape[1:]
        num_classes = len(np.unique(y_train))

        cnn_model = create_cnn(input_shape, num_classes)
        cnn_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

        history = cnn_model.fit(X_train_images_expanded, y_train, validation_data=(X_test_images_expanded, y_test), epochs=10, batch_size=32)

        output_dir = os.path.join('baseline_cnn_classifier_outputs', data_type, 'CNN')
        if not os.path.exists(output_dir):
            os.makedirs(output_dir)

        cnn_model.save(os.path.join(output_dir, 'trained_cnn_classifier.h5'))

        y_prob = cnn_model.predict(X_test_images_expanded)
        plot_roc_curve(y_test, y_prob, output_dir)

        plot_images_and_save(X_train_images_expanded, output_dir, num_images=3, file_name_prefix='train')


In [64]:
data_paths = [#'methylation_1.csv',
              '/content/drive/MyDrive/rnaseq/rnaseq_1.csv',
              '/content/drive/MyDrive/microRNA/microRNA_1.csv'
              ]
main(data_paths)

Columns after determining data type: Index(['1', '2', '3', '4', '5', '6', '7', '8', '9', '10',
       ...
       '22319', '22320', '22321', '22322', '22323', '22324', '22325', '22326',
       '22327', 'age'],
      dtype='object', length=22328)
Index(['1', '2', '3', '4', '5', '6', '7', '8', '9', '10',
       ...
       '22319', '22320', '22321', '22322', '22323', '22324', '22325', '22326',
       '22327', 'age'],
      dtype='object', length=22328)
[0 1 2 3]
About to shuffle
[0 1 2 3]
                     1    2     3     4     5     6     7     8     9    10
donor_id                                                                   
H376.XI.50_STC    28.0  1.0  43.0  19.0   9.0  15.0  19.0  49.0  47.0  38.0
H376.XI.53_OFC    20.0  2.0  47.0  24.0  10.0  14.0  19.0  39.0  40.0  31.0
H376.X.50_CBC     27.0  0.0  50.0  31.0  12.0  18.0  24.0  48.0  43.0  44.0
H376.VIII.52_S1C  33.0  1.0  25.0   7.0   1.0  21.0  17.0  41.0  22.0  18.0
H376.IV.54_V1C    40.0  1.0  48.0  24.0  11.0   5.0   

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


4/4 [==============================] - 1s 278ms/step
Class 0 ROC AUC: 1.00
Class 1 ROC AUC: 1.00
Class 2 ROC AUC: 0.99
Class 3 ROC AUC: 0.99
Macro-average ROC AUC: 0.99
Images saved to baseline_cnn_classifier_outputs/rnaseq/CNN/train_images.png
Columns after determining data type: Index(['hsa-miR-26a-5p', 'hsa-miR-181a-5p', 'hsa-miR-143-3p', 'hsa-let-7a-5p',
       'hsa-miR-9-5p', 'hsa-miR-3182', 'hsa-miR-99b-5p', 'hsa-miR-30a-5p',
       'hsa-miR-27b-3p', 'hsa-miR-191-5p',
       ...
       'hsa-miR-4653-5p', 'hsa-miR-4264', 'hsa-miR-3119', 'hsa-miR-4330',
       'hsa-miR-4318', 'hsa-miR-4279', 'hsa-miR-3689f', 'hsa-miR-4291',
       'donor_name', 'age'],
      dtype='object', length=1863)
Index(['hsa-miR-26a-5p', 'hsa-miR-181a-5p', 'hsa-miR-143-3p', 'hsa-let-7a-5p',
       'hsa-miR-9-5p', 'hsa-miR-3182', 'hsa-miR-99b-5p', 'hsa-miR-30a-5p',
       'hsa-miR-27b-3p', 'hsa-miR-191-5p',
       ...
       'hsa-miR-4653-5p', 'hsa-miR-4264', 'hsa-miR-3119', 'hsa-miR-4330',
       'hsa-miR-43

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


2/2 [==============================] - 1s 140ms/step
Class 0 ROC AUC: 0.56
Class 1 ROC AUC: 0.74
Class 2 ROC AUC: 0.85
Class 3 ROC AUC: 0.87
Macro-average ROC AUC: 0.76
Images saved to baseline_cnn_classifier_outputs/microRNA/CNN/train_images.png
